In [78]:
import gym
import collections
from torch.utils.tensorboard import SummaryWriter

In [79]:
ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
TEST_EPISODES = 20

In [80]:
e = gym.make(ENV_NAME)
e.reset()
print(e.reset())
print(e.step(e.action_space.sample()))

(0, {'prob': 1})
(0, 0.0, False, False, {'prob': 0.3333333333333333})


In [81]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state, _ = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)
    
    # used for gathering random experience from enviroment
    # update the reward and transition tables  
    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _, _ = self.env.step(action)
            # print(self.state, action, new_state)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            if is_done:
                self.state, _ = self.env.reset() 
            else: 
                self.state = new_state
            # print(self.state)
    
    # calculate the value of action from the state using our transition, reward, amd values tables      
    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            val = reward + GAMMA * self.values[tgt_state]
            action_value += (count / total) * val
        return action_value
    
    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action
    
    def play_episode(self, env):
        total_reward = 0.0
        state, _ = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward 
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
    
    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [
                self.calc_action_value(state, action) for action in range(self.env.action_space.n)
            ]
            self.values[state] = max(state_values)
        

In [82]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter(comment='-v-iteration')

In [83]:
iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    agent.play_n_random_steps(100)
    agent.value_iteration()
    
    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
        
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (
            best_reward, reward
        ))
        
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
writer.close()

Best reward updated 0.000 -> 0.300
Best reward updated 0.000 -> 0.150
Best reward updated 0.000 -> 0.050
Best reward updated 0.000 -> 0.400
Best reward updated 0.000 -> 0.400
Best reward updated 0.000 -> 0.450
Best reward updated 0.000 -> 0.350
Best reward updated 0.000 -> 0.700
Best reward updated 0.000 -> 0.700
Best reward updated 0.000 -> 0.900
Solved in 13 iterations!
Best reward updated 0.000 -> 0.750
Best reward updated 0.000 -> 0.550
Best reward updated 0.000 -> 0.650
Best reward updated 0.000 -> 0.750
Best reward updated 0.000 -> 0.800
Best reward updated 0.000 -> 0.550
Best reward updated 0.000 -> 0.600
Best reward updated 0.000 -> 0.650
Best reward updated 0.000 -> 0.750
Best reward updated 0.000 -> 0.550
Best reward updated 0.000 -> 0.500
Best reward updated 0.000 -> 0.700
Best reward updated 0.000 -> 0.750
Best reward updated 0.000 -> 0.800
Best reward updated 0.000 -> 0.800
Best reward updated 0.000 -> 0.500
Best reward updated 0.000 -> 0.800
Best reward updated 0.000 -> 0

KeyboardInterrupt: 